In [ ]:
import nltk

In [ ]:
import pandas as pd

from nltk.corpus import stopwords 
nltk.download('stopwords')
sw=stopwords.words("turkish")

In [ ]:
df=pd.read_excel("C:/Users/USER/Desktop/esra/Veri_Bilimi/ChatBot/metin_madenciligi/Yeni_klasor/soru_cevap.xlsx")
df

In [ ]:
!pip install jpype1

In [ ]:
from jpype import JClass, getDefaultJVMPath, shutdownJVM, startJVM, JString
import os
import string

In [ ]:
ZEMBEREK_PATH = r"/C:/Users/USER/Desktop/esra/Veri_Bilimi/ChatBot/metin_madenciligi/zemberek-full"
startJVM(getDefaultJVMPath(), '-ea', '-Djava.class.path=%s' % (ZEMBEREK_PATH))
DATA_PATH = r"/C:/Users/USER/Desktop/esra/Veri_Bilimi/ChatBot/metin_madenciligi/data"

In [ ]:
TurkishTokenizer: JClass = JClass("zemberek.tokenization.TurkishTokenizer")
TurkishMorphology: JClass = JClass("zemberek.morphology.TurkishMorphology")
TurkishSentenceNormalizer: JClass = JClass(
    "zemberek.normalization.TurkishSentenceNormalizer"
)
Paths: JClass = JClass("java.nio.file.Paths")
morphology = TurkishMorphology.createWithDefaults()
tokenizer = TurkishTokenizer.DEFAULT
normalizer = TurkishSentenceNormalizer(
    TurkishMorphology.createWithDefaults(),
    Paths.get(str(os.path.join(DATA_PATH, "normalization"))),
    Paths.get(str(os.path.join(DATA_PATH, "lm", "lm.2gram.slm"))),
)

In [ ]:
import re

def clean_data(data,stopwords=False):
    data=data.lower()                  #küçük harf
    data=re.sub(r"[^\w\s]","",data)    #noktalama işaretlerinin silinmesi
    data=re.sub(r"[\d]","",data)       #sayıların silinmesi

    liste=[] 
    for word in data.split():
        #Word analysis
        analysis_list = [morphology.analyzeAndDisambiguate(JString(word)).bestAnalysis()[0]]
        #Lemmatization --> Mastar ekli halleri
        #lemm_text = [str(analysis.getDictionaryItem().lemma) for analysis in analysis_list]
        #Normalized Lemmatization --> Mastar ekleri yok.
        norm_lemm_text = [str(analysis.getDictionaryItem().normalizedLemma()) for analysis in analysis_list]
        liste.append(" ".join(norm_lemm_text))
        joiner = " "
        data=joiner.join(liste)


    if stopwords:
        without_sw=""
        for w in data.split():
            if w not in sw:
                without_sw+=" "+w
        data=without_sw

   
    return data
    

def get_cleaned_sentences(df,stopwords=False):    
    cleaned_sentences=[]
    for index,row in df.iterrows():
        #print(index,row)
        cleaned=clean_data(row["Soru"],stopwords);
        cleaned_sentences.append(cleaned);
    
    return cleaned_sentences;

cleaned_sentences=get_cleaned_sentences(df,stopwords=True)
print(cleaned_sentences);

print("\n")

cleaned_sentences_with_stopwords=get_cleaned_sentences(df,stopwords=False)
print(cleaned_sentences_with_stopwords);



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def soru_cevap(soru_embedding,embeddings,df,sorular):
    
    max_sim=-1;
    index_sim=-1;
    for index,SSS_embedding in enumerate(embeddings):
        sim=cosine_similarity(SSS_embedding,soru_embedding)[0][0];
        #print(index, sim, sorular[index])  #bunları bir tabloya ata sırala ve ilk üç değerin dönmesini sağla
        

        if sim>max_sim:
            max_sim=sim;
            index_sim=index;
    

    if max_sim<=0.30:
        print("Bot:Üzgünüm seni anlayamadım!")
        
    else:
        #print("\n")
        #print("Soru: ",soru)
        #print("\n");
        print("Soru: ",df.iloc[index_sim,0]) 
        print("Bot: ", df.iloc[index_sim,1])    #iloc => integer-location  
      
    

In [ ]:
!pip install openpyxl

In [ ]:
from openpyxl import Workbook,load_workbook

In [ ]:
wb = load_workbook("C:/Users/USER/Desktop/esra/Veri_Bilimi/ChatBot/metin_madenciligi/Yeni_klasor/yeni_veri.xlsx")
ws = wb.active

In [ ]:
print(ws["A1"].value)

In [ ]:
flag=True
print("Bot:Merhaba sana yardımcı olmaya çalışacağım. Sohbetten çıkmak için 'bye' yazabilirsin.")
while (flag==True):
    kullanici=input()
    if(kullanici!="bye"):
        liste=[]
        liste.append(kullanici)
        ws.append(liste)
        wb.save("C:/Users/USER/Desktop/esra/Veri_Bilimi/ChatBot/metin_madenciligi/Yeni_klasor/yeni_veri.xlsx")
        soru=clean_data(kullanici,stopwords=True)
        print(soru)
        TfidfVec = TfidfVectorizer()
        tfidf = TfidfVec.fit_transform(cleaned_sentences)
        soru_embedding = TfidfVec.transform(soru.split(","))
        soru_cevap(soru_embedding,tfidf,df,cleaned_sentences)
    elif (kullanici=="bye"):
        flag=False
        print("Bot: Bye..")
        